<a href="https://colab.research.google.com/github/margazadasha/Machine-learning-ML/blob/main/%D0%9C%D0%B0%D1%80%D0%B3%D0%B0%D0%B7%D0%B0_4_6%D0%A4%D0%86%D0%A2_%D0%9F%D0%A0%E2%84%967.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторна робота №7
Рекомендаційні системи.
#Завдання 1. Робота з датасетом ml-100k бібліотеки Surprise.

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357296 sha256=01bc123545e096a599c4e35dda6aecbb96533606dd9790a87be313a4b640d63c
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [7]:
#Вивід перших рядків датасету.
from surprise import Dataset, Reader
from surprise import SVD, CoClustering
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')
print("Перші 8 рядків датасету:")
for row in data.raw_ratings[:8]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 8 рядків датасету:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)
Користувач 298 оцінив фільм 474 на 4.0 балів (часова позначка: 884182806)
Користувач 115 оцінив фільм 265 на 2.0 балів (часова позначка: 881171488)
Користувач 253 оцінив фільм 465 на 5.0 балів (часова позначка: 891628467)


In [11]:
trainset, testset = train_test_split(data, test_size=0.2)

#Параметри для підбору через GridSearchCV для SVD.
param_grid_svd = {
    'n_factors': [65, 100],
    'lr_all': [0.005, 0.009],
    'reg_all': [0.02, 0.07]
}

#Параметри для підбору через GridSearchCV для CoClustering.
param_grid_coclustering = {
    'n_cltr_u': [2, 3],
    'n_cltr_i': [2, 3],
    'n_epochs': [12, 20]
}

#Пошук кращих параметрів для SVD.
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

#Пошук кращих параметрів для CoClustering.
grid_search_coclustering = GridSearchCV(CoClustering, param_grid_coclustering, measures=['rmse', 'mae'], cv=5)
grid_search_coclustering.fit(data)

In [12]:
best_svd = grid_search_svd.best_estimator['mae']
best_coclustering = grid_search_coclustering.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для CoClustering: {grid_search_coclustering.best_score['mae']}")

#Вибір моделі з найменшим MAE.
if grid_search_svd.best_score['mae'] < grid_search_coclustering.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_coclustering
    print("Вибраний алгоритм: CoClustering")

Найкращий MAE для SVD: 0.724422276873067
Найкращий MAE для CoClustering: 0.7476395303922417
Вибраний алгоритм: SVD


In [13]:
best_model.fit(trainset)

#Вивід рекомендацій для конкретного користувача.
user_id = str(253)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

#Отримання рекомендацій.
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

#Прогнозування рейтингів для нерецензованих фільмів.
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

#Вивід 10 найкращих фільмів.
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 253: 74
Топ-10 фільмів, рекомендованих для користувача:
Фільм 174 з прогнозованим рейтингом 4.66
Фільм 178 з прогнозованим рейтингом 4.65
Фільм 963 з прогнозованим рейтингом 4.62
Фільм 1449 з прогнозованим рейтингом 4.58
Фільм 483 з прогнозованим рейтингом 4.56
Фільм 173 з прогнозованим рейтингом 4.49
Фільм 251 з прогнозованим рейтингом 4.47
Фільм 357 з прогнозованим рейтингом 4.47
Фільм 651 з прогнозованим рейтингом 4.46
Фільм 408 з прогнозованим рейтингом 4.45


Висновок: В результаті виконання першого завдання була успішно реалізована рекомендаційна система на базі датасету ml-100k за допомогою бібліотеки Surprise. Завантаження даних і первинний аналіз дозволили ознайомитися зі структурою набору даних, після чого були обрані і реалізовані два алгоритми: SVD та CoClustering. Застосування крос-валідації допомогло оптимізувати параметри моделей, а вибір найкращого алгоритму за середньою абсолютною помилкою (MAE) забезпечив точність прогнозів. В результаті, система змогла згенерувати персоналізовані рекомендації фільмів для конкретного користувача.

#Завдання 2. Побудова власної рекомендаційної системи.

In [29]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

#Завантаження вбудованого датасету ml-100k.
data = Dataset.load_builtin('ml-100k')
reader = Reader(line_format='user item rating timestamp', sep='\t')

#Поділ даних на навчальну та тестову вибірки.
trainset, testset = train_test_split(data, test_size=0.2)

In [30]:
#Реалізація власного алгоритму.
from surprise import AlgoBase, PredictionImpossible

class MyOwnAlgorithm(AlgoBase):
    def __init__(self, sim_options={}, bsl_options={}):
        AlgoBase.__init__(self, sim_options=sim_options, bsl_options=bsl_options)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()
        return self

    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible("Користувач і/або фільм невідомі.")

        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print("Трьома найближчими сусідами користувача", str(u), "є:")
        for v, sim_uv in neighbors[:3]:
            print(f"користувач {v} з подібністю {sim_uv:1.2f}")

        bsl = self.trainset.global_mean + self.bu[u] + self.bi[i]
        return bsl

In [31]:
#Оцінка системи.

#Ініціалізація та навчання алгоритму.
algo = MyOwnAlgorithm()
algo.fit(trainset)

#Прогнозування на тестовій вибірці.
predictions = algo.test(testset)

#Оцінка точності.
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse:.2f}')

Streaming output truncated to the last 5000 lines.
користувач 796 з подібністю 0.52
Трьома найближчими сусідами користувача 341 є:
користувач 713 з подібністю 0.55
користувач 73 з подібністю 0.49
користувач 731 з подібністю 0.40
Трьома найближчими сусідами користувача 107 є:
користувач 509 з подібністю 1.00
користувач 66 з подібністю 1.00
користувач 230 з подібністю 1.00
Трьома найближчими сусідами користувача 165 є:
користувач 267 з подібністю 0.80
користувач 574 з подібністю 0.67
користувач 819 з подібністю 0.64
Трьома найближчими сусідами користувача 135 є:
користувач 796 з подібністю 1.00
користувач 763 з подібністю 1.00
користувач 550 з подібністю 0.88
Трьома найближчими сусідами користувача 406 є:
користувач 295 з подібністю 0.82
користувач 464 з подібністю 0.78
користувач 847 з подібністю 0.71
Трьома найближчими сусідами користувача 426 є:
користувач 731 з подібністю 0.65
користувач 455 з подібністю 0.62
користувач 563 з подібністю 0.56
Трьома найближчими сусідами користувача 16

In [33]:
from collections import defaultdict

#Отримання рейтингів з навчальної вибірки.
trainset = algo.trainset
user_item_dict = defaultdict(list)

#Збір всіх фільми, які оцінили користувачі.
for uid, iid, rating in trainset.all_ratings():
    user_item_dict[trainset.to_raw_uid(uid)].append(trainset.to_raw_iid(iid))

#Вибір перших доступних користувачів та фільмів.
user_id = list(user_item_dict.keys())[0]  #Перший користувач.
item_id = user_item_dict[user_id][0]  #Перший фільм, оцінений цим користувачем.

#Отримання прогнозу.
try:
    prediction = algo.estimate(trainset.to_inner_uid(user_id), trainset.to_inner_iid(item_id))
    print(f'Рейтинг користувача {user_id} фільму під номером {item_id}: {prediction:.2f}')
except PredictionImpossible:
    print("Передбачення неможливе для цієї пари користувач-фільм.")

Трьома найближчими сусідами користувача 0 є:
користувач 0 з подібністю 1.00
користувач 409 з подібністю 1.00
користувач 625 з подібністю 0.86
Рейтинг користувача 837 фільму під номером 628: 3.16


Висновок: В результаті виконання другого завдання була побудована власна рекомендаційна система, яка була реалізована з використанням базового класу AlgoBase з бібліотеки Surprise. Створена система продемонструвала свою здатність до прогнозування оцінок на основі схожості між користувачами, а також базових оцінок. Проведена оцінка підтвердила ефективність алгоритму у створенні рекомендацій, а також можливість адаптації до індивідуальних уподобань користувачів. В результаті було успішно отримано рекомендації фільмів для конкретного користувача.